In [30]:
# importo le librerie necessarie 
import matplotlib.pyplot as plt   
import seaborn as sns            
import pandas as pd    

In [31]:
df = pd.read_csv('CarsDatasets2025.csv', encoding='latin-1')

# Controllo dei valori mancanti per ogni colonna
print("Valori mancanti per colonna ")
print(df.isnull().sum())  # isnull() crea un booleano True/False per i NaN, sum() conta i valori mancanti

# --- INFO GENERALI SUL DATASET ---
print("\n--- INFO DEL DATASET ---")
print(df.info())  # Mostra tipo di dato, numero righe, valori nulli, memoria usata


Valori mancanti per colonna 
Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

--- INFO DEL DATASET ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   object
 3   CC/Battery Capacity        1215 non-null   object
 4   HorsePower                 1218 non-null   object
 5   Total Speed                1218 non-null   object
 6   Performance(0 - 100 )KM/H  1212 non-nul

In [32]:
import pandas as pd

df = pd.read_csv('CarsDatasets2025.csv', encoding='latin-1')
print(df.columns)
# Controllo valori mancanti
print("Valori mancanti per colonna:")
print(df.isnull().sum())

# Info generali
print("\n--- INFO DEL DATASET ---")
print(df.info())
target_column = 'Cars Prices'  
y = df[target_column].values
X = df.drop(columns=[target_column])

# Solo colonne numeriche (opzionale)
X = X.select_dtypes(include='number')


Index(['Company Names', 'Cars Names', 'Engines', 'CC/Battery Capacity',
       'HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices',
       'Fuel Types', 'Seats', 'Torque'],
      dtype='object')
Valori mancanti per colonna:
Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

--- INFO DEL DATASET ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   o

In [33]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.preprocessing import StandardScaler

import pandas as pd
from sklearn.preprocessing import StandardScaler

target_column = 'Cars Prices'  
def range_to_mean(val):
    if pd.isna(val):
        return np.nan
    # Rimuovi simboli $ e spazi
    val = val.replace('$', '').replace(',', '').strip()
    # Sostieni sia '-' che '/' come separatori
    if '-' in val:
        parts = val.split('-')
    elif '/' in val:
        parts = val.split('/')
    else:
        parts = [val]
    try:
        numbers = [float(p.strip()) for p in parts]
        return np.mean(numbers)
    except:
        return np.nan  # se non è convertibile, diventa NaN

# Applica
df['Cars Prices Numeric'] = df['Cars Prices'].apply(range_to_mean)
df = df.dropna(subset=['Cars Prices Numeric'])
y = df['Cars Prices Numeric'].values
y = df['Cars Prices Numeric'].values

# Seleziona colonne numeriche + encode categoriche
X = df.drop(columns=[target_column])
X = pd.get_dummies(X, drop_first=True)  # trasforma eventuali stringhe in numerico

# Controllo
print(X.columns)
print(X.shape)

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = scaler.fit_transform(X)
def evaluate_regression_model(model, X, y, setting_name):
    mse_scores = -cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    rmse_scores = np.sqrt(mse_scores)
    r2_scores = cross_val_score(model, X, y, cv=3, scoring='r2', n_jobs=-1)
    
    return {
        "model": model.__class__.__name__,
        "setting": setting_name,
        "MSE_mean": np.mean(mse_scores),
        "MSE_std": np.std(mse_scores),
        "RMSE_mean": np.mean(rmse_scores),
        "RMSE_std": np.std(rmse_scores),
        "R2_mean": np.mean(r2_scores),
        "R2_std": np.std(r2_scores)
    }

results = []

results.append(evaluate_regression_model(XGBRegressor(n_estimators=50, max_depth=4, learning_rate=0.05, random_state=42),
                                         X_scaled, y, "XGBoost 50 trees"))
results.append(evaluate_regression_model(RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42),
                                         X_scaled, y, "RandomForest 50 trees"))
results.append(evaluate_regression_model(SVR(kernel='rbf', C=10, epsilon=0.1),
                                         X_scaled, y, "SVR RBF"))
results.append(evaluate_regression_model(LinearRegression(),
                                         X_scaled, y, "LinearRegression"))

import pandas as pd
df_results = pd.DataFrame(results)
print(df_results)


Index(['Cars Prices Numeric', 'Company Names_AUDI', 'Company Names_Acura',
       'Company Names_BENTLEY', 'Company Names_BMW', 'Company Names_Bugatti',
       'Company Names_Cadillac', 'Company Names_Chevrolet',
       'Company Names_FERRARI', 'Company Names_Ford',
       ...
       'Torque_881 Nm', 'Torque_885 Nm', 'Torque_893 Nm', 'Torque_90 Nm',
       'Torque_900 Nm', 'Torque_93 Nm', 'Torque_94 Nm', 'Torque_95 Nm',
       'Torque_950 Nm', 'Torque_967 Nm'],
      dtype='object', length=2921)
(1213, 2921)
                   model                setting      MSE_mean       MSE_std  \
0           XGBRegressor       XGBoost 50 trees  2.309532e+11  3.125094e+11   
1  RandomForestRegressor  RandomForest 50 trees  2.598278e+11  3.639813e+11   
2                    SVR                SVR RBF  5.159263e+11  6.318669e+11   
3       LinearRegression       LinearRegression  1.710040e+09  2.418362e+09   

       RMSE_mean       RMSE_std   R2_mean    R2_std  
0  329374.186527  349951.168410  0.7